In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import gmaps

# Import API key
from config import g_key

In [3]:
vacation = pd.read_csv('../WeatherPy/output_data/cities.csv')
len(vacation)

563

In [15]:
ideal_temp_min = vacation.loc[vacation["max temperature"] >=71]
ideal_temp_max = ideal_temp_min.loc[ideal_temp_min["max temperature"]<78]
ideal_wind = ideal_temp_max.loc[ideal_temp_max['wind speed (mph)']<10]
ideal_vacation = ideal_wind.loc[ideal_wind['cloudiness (%)']<10]
len(ideal_vacation)

9

In [16]:
# Create a hotel_df
hotel_search_df = ideal_vacation.loc[:,["city","country", "latitude", "longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_search_df["Hotel Name"] = ""

# Display the result
hotel_search_df

,city,country,latitude,longitude,Hotel Name
15,laminusa,BR,-16.3578,-49.4961,
111,beberon,CN,31.8128,121.6561,
122,hyde,AM,40.0000,45.0000,
252,miercurea-ciuc,AM,40.0275,44.3670,
287,mazinde,ES,42.5216,-7.5142,
361,stenlille,US,34.0454,-81.1293,
415,aizkraukle,US,32.5160,-93.7321,
416,ome,PT,41.3495,-8.4015,
454,levice,IT,40.7272,8.5603,


In [17]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 3,
          "key" : g_key}

In [18]:
for index, row in hotel_search_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["latitude"]
    lng = row["longitude"]
    city_name = row["city"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel (within 3 mi range) in {city_name} is {results[0]['name']}.")
        hotel_search_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 15: laminusa.
Missing field/result... skipping.
------------
Retrieving Results for Index 111: beberon.
Closest hotel (within 3 mi range) in beberon is Hanting Express Qidong Jianghai Middle Road.
------------
Retrieving Results for Index 122: hyde.
Missing field/result... skipping.
------------
Retrieving Results for Index 252: miercurea-ciuc.
Missing field/result... skipping.
------------
Retrieving Results for Index 287: mazinde.
Closest hotel (within 3 mi range) in mazinde is Hotel Ribeira Sacra.
------------
Retrieving Results for Index 361: stenlille.
Missing field/result... skipping.
------------
Retrieving Results for Index 415: aizkraukle.
Missing field/result... skipping.
------------
Retrieving Results for Index 416: ome.
Missing field/result... skipping.
------------
Retrieving Results for Index 454: levice.
Closest hotel (within 3 mi range) in levice is Piazza Azuni 18.
------------
-------End of Search-------


In [19]:
hotel_search_df

,city,country,latitude,longitude,Hotel Name
15,laminusa,BR,-16.3578,-49.4961,
111,beberon,CN,31.8128,121.6561,Hanting Express Qidong Jianghai Middle Road
122,hyde,AM,40.0000,45.0000,
252,miercurea-ciuc,AM,40.0275,44.3670,
287,mazinde,ES,42.5216,-7.5142,Hotel Ribeira Sacra
361,stenlille,US,34.0454,-81.1293,
415,aizkraukle,US,32.5160,-93.7321,
416,ome,PT,41.3495,-8.4015,
454,levice,IT,40.7272,8.5603,Piazza Azuni 18


In [20]:
#CONFIGURE GMAPS
gmaps.configure(api_key=g_key)

In [21]:
# Store 'Lat' and 'Lng' into  locations 
vacation = vacation.dropna()
locations = vacation[["latitude", "longitude"]].astype(float)

# humity to plot heat map
humidity = vacation["humidity (%)"].astype(float)

In [22]:
# humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Convert nearest hotel to list and filter for null values (where hotel was not found within range)

locations_hotels = hotel_search_df.loc[hotel_search_df["Hotel Name"] !=""]
hotels = locations_hotels["Hotel Name"].tolist()

marker_locations = locations_hotels[['latitude', 'longitude']]

In [26]:
# Create a marker_layer using the poverty list to fill the info box and combine both maps together
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Near Hotels: {hotel}" for hotel in hotels])
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))